In [47]:
import torch
import cv2
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%run helpers.ipynb
%run NN.ipynb
np.set_printoptions(precision=2)
torch.manual_seed(42)
np.random.seed(42)

In [48]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Hyper parameters

In [49]:
canvas_width = 51
canvas_height= 51
center = np.array([(canvas_height-1) / 2, (canvas_width-1) / 2])
num_drawings = 1000
num_samples = 1000
bins = np.arange(0,21)

ts = np.array([0.17,0.2,0.08,0.1])

# Generate the dataset

In [50]:
canvass_train = np.zeros([num_drawings,canvas_height, canvas_width])
canvass_test = np.zeros([num_drawings,canvas_height, canvas_width])

In [51]:
for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_train[i] = draw_polygon(canvass_train[2],3,5)
    else:
        canvass_train[i] = draw_ellipse(canvass_train[i],10,15)

for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_test[i] = draw_polygon(canvass_test[2],3,5)
    else:
        canvass_test[i] = draw_ellipse(canvass_test[i],10,15)

# Get the length vectors

In [52]:
lengthss_train = get_length_encoding(canvass_train,num_samples,bins)
lengthss_test = get_length_encoding(canvass_test,num_samples,bins)

# Turn image into graph

In [53]:
lap_mats_train = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_train))
lap_mats_test = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_test))

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


# Compute eigenvalues & eigenvectors &hks

In [54]:
evals_train,Us_train = lap_mat_to_eigen(lap_mats_train)
hkss_train = batch_heat_kernel_signature(ts, evals_train, Us_train)

evals_test,Us_test = lap_mat_to_eigen(lap_mats_test)
hkss_test = batch_heat_kernel_signature(ts, evals_test, Us_test)

# Set up Neural Network

In [55]:
class HeatKernelSignatureNet(nn.Module):
    #takes in image (51 x 51), returns heat kernel signature at [0.17,0.47,0.58,0.81]

    def __init__(self,image_width,image_height):
        super().__init__()
        self.image_width = image_width
        self.image_height = image_height

        self.MLP = MLP(20,4,10,100)


    def forward(self, x):

        x = torch.flatten(x,start_dim=1,end_dim=-1)
        x = self.MLP(x)

        return x

In [56]:
hksNet = HeatKernelSignatureNet(51,51)
hksNet.to(device)

HeatKernelSignatureNet(
  (MLP): MLP(
    (model): Sequential(
      (fc1): Linear(in_features=20, out_features=100, bias=True)
      (tanh1): Tanh()
      (fc2): Linear(in_features=100, out_features=100, bias=True)
      (tanh2): Tanh()
      (fc3): Linear(in_features=100, out_features=100, bias=True)
      (tanh3): Tanh()
      (fc4): Linear(in_features=100, out_features=100, bias=True)
      (tanh4): Tanh()
      (fc5): Linear(in_features=100, out_features=100, bias=True)
      (tanh5): Tanh()
      (fc6): Linear(in_features=100, out_features=100, bias=True)
      (tanh6): Tanh()
      (fc7): Linear(in_features=100, out_features=100, bias=True)
      (tanh7): Tanh()
      (fc8): Linear(in_features=100, out_features=100, bias=True)
      (tanh8): Tanh()
      (fc9): Linear(in_features=100, out_features=100, bias=True)
      (tanh9): Tanh()
      (fc10): Linear(in_features=100, out_features=4, bias=True)
    )
  )
)

In [57]:
lengthss_train_torch = torch.tensor(lengthss_train).to(device).float()
lengthss_train_torch = torch.reshape(lengthss_train_torch,[num_drawings,1,bins.shape[0]-1])

lengthss_test_torch = torch.tensor(lengthss_test).to(device).float()
lengthss_test_torch = torch.reshape(lengthss_test_torch,[num_drawings,1,bins.shape[0]-1])

hkss_train_torch = torch.tensor(hkss_train).to(device).float()
hkss_test_torch = torch.tensor(hkss_test).to(device).float()

In [58]:
print(lengthss_test_torch.shape)

torch.Size([1000, 1, 20])


In [59]:
learning_rates = [0.005] * 2000 + [0.0005] * 2000 + [0.00005] * 2000

In [60]:
for i in range(len(learning_rates)):
    opt = torch.optim.AdamW(list(hksNet.parameters()),learning_rates[i])
    opt.zero_grad()

    pred_hks = hksNet(lengthss_train_torch)
    loss = torch.mean(( (pred_hks - hkss_train_torch) / hkss_train_torch )**2)
    loss.backward()
    opt.step()

    if i % 10 == 0:
        print(i,loss.item())

0 0.9343060851097107
10 0.13358964025974274
20 0.12981946766376495
30 0.1494787484407425
40 0.1610633283853531
50 0.14798825979232788
60 0.1458957940340042
70 0.12013617157936096
80 0.11220108717679977
90 0.08782593160867691
100 0.09128700941801071
110 0.0942261591553688
120 0.0867563933134079
130 0.09370474517345428
140 0.08714015781879425
150 0.07836836576461792
160 0.07948675751686096
170 0.10060915350914001
180 0.12274070084095001
190 0.2086097151041031
200 0.02894633822143078
210 0.015453608706593513
220 0.06403046101331711
230 0.05766848847270012
240 0.010251143015921116
250 0.018321415409445763
260 0.08296182006597519
270 0.013676566071808338
280 0.1820083111524582
290 0.10142886638641357
300 0.030531415715813637
310 0.10880973935127258
320 0.07113993912935257
330 0.1334071010351181
340 0.11470884829759598
350 0.05220314487814903
360 0.06626588851213455
370 0.05392753705382347
380 0.059945978224277496
390 0.012260709889233112
400 0.01823265664279461
410 0.006026335526257753
420 

# Test

In [61]:
pred_hks_test = hksNet(lengthss_test_torch)
loss_test = torch.mean(( ( pred_hks_test - hkss_test_torch) / hkss_test_torch )**2)
print(loss_test)

tensor(0.0003, device='cuda:0', grad_fn=<MeanBackward0>)


# Randomly sample a few to see what it looks like

In [62]:
print(pred_hks_test[15])
print(hkss_test_torch[15])

tensor([3.8888, 5.0277, 1.8231, 2.1581], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.7095, 4.7786, 1.7998, 2.1027], device='cuda:0')


In [63]:
print(pred_hks_test[35])
print(hkss_test_torch[35])

tensor([3.8663, 5.0301, 1.8338, 2.1553], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.9013, 5.0584, 1.8487, 2.1724], device='cuda:0')


In [64]:
print(pred_hks_test[60])
print(hkss_test_torch[60])

tensor([3.8791, 5.0155, 1.8284, 2.1573], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.6584, 4.7068, 1.7852, 2.0823], device='cuda:0')


In [65]:
print(pred_hks_test[96])
print(hkss_test_torch[96])

tensor([3.8414, 4.9720, 1.8138, 2.1424], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.7638, 4.8598, 1.8125, 2.1211], device='cuda:0')
